In [ ]:
###############################################################################
import aliases

In [ ]:
from everest.ptolemaic.diict import Kwargs

from everest.algebraic.floatt import Floatt
from everest.algebraic.brace import Brace

from everest.formulaic.ode import ODEModel

In [ ]:
@ODEModel
def lorenz(
        index: Floatt[0.:],
        state: Brace[Kwargs(x=Floatt, y=Floatt, z=Floatt)],
        sigma: Floatt,
        rho: Floatt,
        beta: Floatt,
        /):
    x, y, z = state
    return (
        sigma * (y - x),
        rho * x - y - x * z,
        x * y - beta * z,
        )

lorenz

In [ ]:
lorenz.plex = 1

In [ ]:
traverse = lorenz[
    (10., 28., 2.667),
    (0., 1., 1.05),
    0.:100.,
    ]
traverse

In [ ]:
traverse.line.table

In [ ]:
data = traverse.solve()

In [ ]:
vertices = data.y.T
vertices.shape

In [ ]:
import numpy as np
vertices = data.y.T
ts = data.t
dts = np.diff(ts)
diffs = np.diff(vertices, axis=0)
speeds = np.linalg.norm(diffs, axis=1) / dts
accels = np.diff(speeds) / dts[1:]

In [ ]:
import lavavu

lv = lavavu.Viewer()
lv["background"] = "black" #Set the plot background colour
lv["axis"] = False
lv["border"] = False

cmap = lavavu.cubehelix(samples=128, rot=1., sat=2., start=-1.)[1:-1]
lines = lv.lines(link=True)
lines.vertices(vertices[1:])
lines.colourmap(cmap)
lines.values(accels)

lv.reload()
lv.display(resolution=(600, 600))

In [ ]:
import lavavu

lv = lavavu.Viewer()
lv["background"] = "black" #Set the plot background colour
lv["axis"] = False
lv["border"] = False

lines = lv.lines(link=True)
lines.vertices(np.dstack((ts, vertices[..., 0])))
lines.colourmap('red')
lines.values(1)

lines = lv.lines(link=True)
lines.vertices(np.dstack((ts, vertices[..., 1])))
lines.colourmap('blue')
lines.values(1)

lines = lv.lines(link=True)
lines.vertices(np.dstack((ts, vertices[..., 2])))
lines.colourmap('green')
lines.values(1)

lv.reload()

lv.display(resolution=(600, 600))